In [9]:
%matplotlib inline


# Interpretable models

The following example shows how to inspect the models which *auto-sklearn*
optimizes over and how to restrict them to an interpretable subset.


In [1]:
from pprint import pprint

import autosklearn.classification
import sklearn.datasets
import sklearn.metrics

## Show available classification models

We will first list all classifiers Auto-sklearn chooses from. A similar
call is available for preprocessors (see below) and regression (not shown)
as well.



In [2]:
from autosklearn.pipeline.components.classification import ClassifierChoice

for name in ClassifierChoice.get_components():
    print(name)

adaboost
bernoulli_nb
decision_tree
extra_trees
gaussian_nb
gradient_boosting
k_nearest_neighbors
lda
liblinear_svc
libsvm_svc
mlp
multinomial_nb
passive_aggressive
qda
random_forest
sgd


## Show available preprocessors



In [12]:
from autosklearn.pipeline.components.feature_preprocessing import (
    FeaturePreprocessorChoice,
)

for name in FeaturePreprocessorChoice.get_components():
    print(name)

densifier
extra_trees_preproc_for_classification
extra_trees_preproc_for_regression
fast_ica
feature_agglomeration
kernel_pca
kitchen_sinks
liblinear_svc_preprocessor
no_preprocessing
nystroem_sampler
pca
polynomial
random_trees_embedding
select_percentile_classification
select_percentile_regression
select_rates_classification
select_rates_regression
truncatedSVD


In [13]:
from autosklearn.pipeline.components.data_preprocessing import (
    DataPreprocessorChoice,
)

for name in DataPreprocessorChoice.get_components():
    print(name)

feature_type


In [14]:
DataPreprocessorChoice.get_components()

OrderedDict([('feature_type',
              autosklearn.pipeline.components.data_preprocessing.feature_type.FeatTypeSplit)])

## Data Loading



In [15]:
X, y = sklearn.datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X, y, random_state=1
)

## Build and fit a classifier

We will now only use a subset of the given classifiers and preprocessors.
Furthermore, we will restrict the ensemble size to ``1`` to only use the
single best model in the end. However, we would like to note that the
choice of which models is deemed interpretable is very much up to the user
and can change from use case to use case.



In [16]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    memory_limit=16384,
    tmp_folder="/tmp/autosklearn_interpretable_models_example_tmp",
    include={
        "classifier": ["decision_tree", "lda", "sgd"],
        "feature_preprocessor": [
            "no_preprocessing",
            "polynomial",
            "select_percentile_classification",
        ],
    },
    ensemble_kwargs={"ensemble_size": 1},
)
automl.fit(X_train, y_train, dataset_name="breast_cancer")

[WARNING] [2024-09-23 18:10:32,166:Client-AutoMLSMBO(1)::breast_cancer] Configuration 176 not found
[WARNING] [2024-09-23 18:10:32,166:Client-AutoMLSMBO(1)::breast_cancer] Configuration 566 not found
[WARNING] [2024-09-23 18:10:32,166:Client-AutoMLSMBO(1)::breast_cancer] Configuration 162 not found
[WARNING] [2024-09-23 18:10:32,166:Client-AutoMLSMBO(1)::breast_cancer] Configuration 476 not found
[WARNING] [2024-09-23 18:10:32,166:Client-AutoMLSMBO(1)::breast_cancer] Configuration 601 not found
[WARNING] [2024-09-23 18:10:32,166:Client-AutoMLSMBO(1)::breast_cancer] Configuration 492 not found
[WARNING] [2024-09-23 18:10:32,166:Client-AutoMLSMBO(1)::breast_cancer] Configuration 440 not found
[WARNING] [2024-09-23 18:10:32,166:Client-AutoMLSMBO(1)::breast_cancer] Configuration 32 not found
[WARNING] [2024-09-23 18:10:32,166:Client-AutoMLSMBO(1)::breast_cancer] Configuration 220 not found
[WARNING] [2024-09-23 18:10:32,166:Client-AutoMLSMBO(1)::breast_cancer] Configuration 332 not found
[

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      ensemble_kwargs={'ensemble_size': 1},
                      include={'classifier': ['decision_tree', 'lda', 'sgd'],
                               'feature_preprocessor': ['no_preprocessing',
                                                        'polynomial',
                                                        'select_percentile_classification']},
                      memory_limit=16384, per_run_time_limit=30,
                      time_left_for_this_task=120,
                      tmp_folder='/tmp/autosklearn_interpretable_models_example_tmp')

## Print the final ensemble constructed by auto-sklearn



In [17]:
pprint(automl.show_models(), indent=4)

{   110: {   'balancing': Balancing(random_state=1),
             'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7031b09548e0>,
             'cost': 0.007092198581560294,
             'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7031b0a5bf40>,
             'ensemble_weight': 1.0,
             'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7031b0954e80>,
             'model_id': 110,
             'rank': 1,
             'sklearn_classifier': SGDClassifier(alpha=0.013119453170385034, average=True,
              eta0=0.00011933117699148683, learning_rate='invscaling',
              loss='log', max_iter=1024, random_state=1,
              tol=1.3749800289585616e-05, warm_start=True)}}


In [18]:
pprint(automl, indent=4)

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      ensemble_kwargs={'ensemble_size': 1},
                      include={'classifier': ['decision_tree', 'lda', 'sgd'],
                               'feature_preprocessor': ['no_preprocessing',
                                                        'polynomial',
                                                        'select_percentile_classification']},
                      memory_limit=16384, per_run_time_limit=30,
                      time_left_for_this_task=120,
                      tmp_folder='/tmp/autosklearn_interpretable_models_example_tmp')


## Get the Score of the final ensemble



In [19]:
predictions = automl.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))

Accuracy score: 0.9440559440559441


In [20]:
automl

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      ensemble_kwargs={'ensemble_size': 1},
                      include={'classifier': ['decision_tree', 'lda', 'sgd'],
                               'feature_preprocessor': ['no_preprocessing',
                                                        'polynomial',
                                                        'select_percentile_classification']},
                      memory_limit=16384, per_run_time_limit=30,
                      time_left_for_this_task=120,
                      tmp_folder='/tmp/autosklearn_interpretable_models_example_tmp')